In [1]:
from model import SimpleView
from dataset import MpalaTreeLiDAR
from torch.utils.data import DataLoader, random_split
from torchvision import transforms
from torch import optim, nn
from tqdm import tqdm
import pc_transform
from util import load_fake_dataset

ids, labels, classes = load_fake_dataset(min_points=500)

dataset = MpalaTreeLiDAR(
    data_path='../data/MpalaForestGEO_LasClippedtoTreePolygons',
    ids=ids,
    labels=labels,
    classes=classes,
    transform=transforms.Compose([
        pc_transform.ToPointCloud(),
        pc_transform.ProjectPointCloud(),
        pc_transform.ExpandChannels(channels=3),
    ]),
)
trainset, testset = random_split(dataset, [0.85, 0.15])
trainloader = DataLoader(trainset, batch_size=64, shuffle=True)

simpleview = SimpleView(num_views=6, num_classes=len(dataset.classes))
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(simpleview.parameters(), lr=0.001, momentum=0.9)
epochs = 100

for _ in tqdm(range(epochs)):
    for x, y in trainloader:
        print('batch done')
        optimizer.zero_grad()
        outputs = simpleview(x)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()
        break
    break

Found 16853 trees
Found only 8033 trees with more than 500 points


  0%|          | 0/100 [00:00<?, ?it/s]

batch done


  0%|          | 0/100 [01:03<?, ?it/s]


In [ ]:
# import wandb
# wandb.init(project='my-test-project')
# wandb.config = {
#   "learning_rate": 0.001,
#   "epochs": 100,
#   "batch_size": 128
# }
# wandb.log({"loss": loss})
# wandb.watch(simpleview)